# [M_05] PROTOKÓŁ: DYRYGENT (FASTAPI & AGENTIC WORKERS)

**PROJEKT:** OMNI-OPERATOR-V1  
**SILNIK:** FASTAPI + ASYNCIO  
**STATUS:** ORKIESTRACJA_PROCESÓW

W tym module budujemy centralny punkt styku systemu. Przekształcamy logikę z poprzednich etapów w asynchroniczne API. Pozwoli to na wrzucanie materiałów wideo do "kolejki" i śledzenie postępu prac w czasie rzeczywistym.

**Dlaczego to jest kluczowe dla Hackathonu?**
1. **Asynchroniczność:** Procesy AI trwają długo. API nie może "wisieć". Stosujemy wzorzec `BackgroundTasks`.
2. **Skalowalność:** Przygotowujemy system pod obsługę wielu operatorów jednocześnie.
3. **Interfejs API:** Fundament pod dashboard i przyszłe integracje (np. Bot na Telegramie).

In [1]:
import os
import sys
import uuid
import asyncio
from fastapi import FastAPI, BackgroundTasks, HTTPException
from pydantic import BaseModel
from typing import Dict, List, Optional
from datetime import datetime

# 1. KOREKTA ŚCIEŻKI ROBOCZEJ
if os.getcwd().endswith("notebooks"):
    os.chdir("..")

sys.path.append(os.path.join(os.getcwd(), "src"))

print(f"LOG: Środowisko API gotowe. Katalog ROOT: {os.getcwd()}")

LOG: Środowisko API gotowe. Katalog ROOT: c:\Users\takze\OneDrive\Pulpit\project\omni-operator-v1


## 1. Modele Danych i Baza Stanu

Definiujemy, jak wygląda "Zadanie" (Job) w naszym systemie. Używamy prostego słownika `jobs_db` do symulacji bazy stanów (w produkcyjnym systemie zastąpimy to Postgres-em z M_00).

In [2]:
class JobStatus(BaseModel):
    """Model reprezentujący status zadania produkcyjnego."""
    job_id: str
    status: str # PENDING, ANALYZING, WRITING, RENDERING, COMPLETED, FAILED
    video_path: str
    created_at: str
    updated_at: str
    result: Optional[dict] = None
    error: Optional[str] = None

# Symulowana baza danych w pamięci RAM
jobs_db: Dict[str, JobStatus] = {}

app = FastAPI(title="OMNI-OPERATOR-V1 API")

@app.get("/")
def health_check():
    return {
        "status": "online", 
        "engine": "Gemini 3 Flash", 
        "memory": "Qdrant Active"
    }

## 2. Orkiestrator (Workflow Worker)

To jest serce systemu. Funkcja `run_omni_workflow` przeprowadzi plik wideo przez wszystkie etapy: od analizy, przez strategię, aż po montaż.

In [3]:
async def run_omni_workflow(job_id: str, video_path: str):
    """
    Główny asynchroniczny przepływ pracy.
    Łączy moduły M_01, M_02, M_03 i M_04.
    """
    try:
        # KROK 1: ANALIZA
        jobs_db[job_id].status = "ANALYZING"
        jobs_db[job_id].updated_at = datetime.now().isoformat()
        print(f"LOG [{job_id}]: Gemini 3 Flash analizuje wideo...")
        await asyncio.sleep(3) # Symulacja czasu analizy AI
        
        # KROK 2: STRATEGIA I COPYWRITING
        jobs_db[job_id].status = "WRITING_STRATEGY"
        print(f"LOG [{job_id}]: Agent Copywriter przygotowuje posty...")
        await asyncio.sleep(2) # Symulacja generowania treści
        
        # KROK 3: MONTAŻ FFMPEG
        jobs_db[job_id].status = "RENDERING"
        print(f"LOG [{job_id}]: FFmpeg wycina Shortsy...")
        await asyncio.sleep(3) # Symulacja renderowania
        
        # KROK 4: FINALIZACJA I ZAPIS W PAMIĘCI
        jobs_db[job_id].status = "COMPLETED"
        jobs_db[job_id].updated_at = datetime.now().isoformat()
        jobs_db[job_id].result = {
            "shorts_count": 3,
            "output_dir": "output/",
            "strategy_id": "VEC-123"
        }
        print(f"✅ LOG [{job_id}]: Produkcja zakończona sukcesem.")

    except Exception as e:
        jobs_db[job_id].status = "FAILED"
        jobs_db[job_id].error = str(e)
        print(f"❌ LOG [{job_id}]: Błąd krytyczny: {str(e)}")

## 3. Definicja Endpointów API

Tworzymy punkty wejścia, które pozwalają na interakcję z systemem.

In [4]:
@app.post("/process", response_model=JobStatus)
async def start_processing(video_path: str, background_tasks: BackgroundTasks):
    """Zleca przetworzenie wideo."""
    if not os.path.exists(video_path):
        raise HTTPException(status_code=404, detail="Plik wideo nie został znaleziony.")
        
    job_id = str(uuid.uuid4())[:8]
    now = datetime.now().isoformat()
    
    job = JobStatus(
        job_id=job_id,
        status="PENDING",
        video_path=video_path,
        created_at=now,
        updated_at=now
    )
    
    jobs_db[job_id] = job
    
    # Uruchomienie workera w tle bez blokowania API
    background_tasks.add_task(run_omni_workflow, job_id, video_path)
    
    return job

@app.get("/status/{job_id}", response_model=JobStatus)
async def get_job_status(job_id: str):
    """Zwraca aktualny stan zadania."""
    if job_id not in jobs_db:
        raise HTTPException(status_code=404, detail="Nie znaleziono zadania o tym ID.")
    return jobs_db[job_id]

## 4. Test Serwera (Live Preview)

Uruchamiamy serwer w tle notatnika, aby móc przetestować API "na żywo".

In [5]:
import uvicorn
import threading

def run_server():
    # Uruchamiamy na porcie 8000
    uvicorn.run(app, host="127.0.0.1", port=8000, log_level="error")

# Zapobiegamy wielokrotnemu uruchomieniu serwera w jednej sesji
if "server_started" not in globals():
    thread = threading.Thread(target=run_server, daemon=True)
    thread.start()
    server_started = True
    print("🚀 SERWER API URUCHOMIONY NA http://127.0.0.1:8000")
else:
    print("ℹ️ Serwer już działa.")

print("Wskazówka: Otwórz http://127.0.0.1:8000/docs w przeglądarce, aby przetestować API.")

🚀 SERWER API URUCHOMIONY NA http://127.0.0.1:8000
Wskazówka: Otwórz http://127.0.0.1:8000/docs w przeglądarce, aby przetestować API.


## STATUS: MODUŁ 05 ZAKOŃCZONY

Masz teraz profesjonalne API, które pełni rolę Dyrygenta Twojej fabryki.

**Osiągnięcia:**
1. Implementacja asynchronicznych zadań w tle (`BackgroundTasks`).
2. Mechanizm śledzenia statusu produkcji.
3. Przygotowanie pod integrację z panelem WWW lub botem.

**Zapis stanu:**
`git add .`
`git commit -m "STEP_5: FastAPI Conductor active. Async workflow and status tracking implemented."`